# Cluster + client
Here we get access tot the cluster and connect a client. Click on manual scale 2. 

In [1]:
import dask_gateway

cluster = dask_gateway.GatewayCluster()
client = cluster.get_client()
cluster

# Install dependencies in workers
Install the r environment in workers and add two dependencies (python-r, r-python).

In [2]:
# conda is a bit slow, so we'll use mamba to install by hand
from dask.distributed import CondaInstall

plugin = CondaInstall(packages=["r-base", "rpy2", "r-reticulate"])

# this is a bit slow
# client.register_worker_plugin(plugin)

In [3]:
# install using a system command
def worker_setup(dask_worker):
    # install R dependencies from github
    import os

    os.system("mamba install -y r-base rpy2 r-reticulate")


use_worker_setup = True
if use_worker_setup:
    client.register_worker_callbacks(worker_setup)

In [4]:
def test_r():
    import rpy2.robjects as robjects
    from rpy2.robjects import conversion, default_converter

    # get pi from R
    with conversion.localconverter(default_converter):
        pi = robjects.r["pi"][0]
    return float(pi)


def test_py():
    import numpy as np

    # get pi from numpy
    pi = np.pi
    return pi

In [5]:
test_r()

ModuleNotFoundError: No module named 'rpy2'

In [6]:
future_r = client.submit(test_r)
future_py = client.submit(test_py)

In [7]:
client.get_worker_logs()

{}